In [ ]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

In [ ]:
viral_reference_fastas_dir = mkpath(joinpath(data_dir, "viral-reference-fastas"))

In [ ]:
run(`$(Mycelia.MAMBA) run --live-stream -n blast makeblastdb -help`)

In [ ]:
blastdb_dir = joinpath(homedir(), "workspace", "blastdb")

In [ ]:
reference_fastas = sort(filter(x -> occursin(r"\.f(na|asta)\.gz$", x), readdir(viral_reference_fastas_dir, join=true)), by=x->filesize(x))

In [ ]:
input_records = Mycelia.open_fastx("/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.fna.gz")
revised_record_iterator = (FASTX.FASTA.Record(first(split(FASTX.description(record), '|')), FASTX.sequence(record)) for record in input_records)
IMGVR_shorted_descriptions = "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.revised.fna.gz"

In [ ]:
Mycelia.write_fasta(outfile = IMGVR_shorted_descriptions, gzipped = true, records = revised_record_iterator)

In [ ]:
for reference_fasta in reference_fastas
    database_title = replace(basename(reference_fasta), r"\.f(na|asta)\.gz" => "")
    blastdb = joinpath(blastdb_dir, database_title)
    @show blastdb
    hits = filter(x -> occursin(blastdb, x), readdir(blastdb_dir, join=true))
    # @show hits
    if isempty(hits)
        @show "make me"
        @time run(pipeline(`gzip -dc $(reference_fasta)`, `$(Mycelia.MAMBA) run --live-stream -n blast makeblastdb -title $(database_title) -in - -dbtype nucl -parse_seqids -out $(blastdb)`))
    else
        @show "done"
    end
end